# Deconstructing a Monet
> While Normandy is only 800km away, the past is a foreign country. 

- toc: true 
- badges: true
- comments: true
- categories: [Computer Vision,Linear Algebra]
- image: images/2020-04-13-Deconstructing-A-Monet/header.jpg

![](../images/2020-04-13-Deconstructing-A-Monet/Monet_-_Sonnenaufgang_bei_Etretat.jpg)

## An introduction

Let's go on an adventure, and try and answer some questions from an incredible painting by Claude Monet called [The Cliff, Étretat, Sunset](https://learn.ncartmuseum.org/artwork/the-cliff-etretat-sunset/).

In particular, I'm interested if we can work out:

1. Where it was painted.
2. When it was painted.

These questions are by no means new, they were addressed by [Don Olson](https://www.smithsonianmag.com/arts-culture/forensic-astronomer-solves-fine-arts-puzzles-116770403) and his [team](https://www.txstate.edu/news/news_releases/news_archive/2014/January-2014/Olson012114.html) from Texas State University].


However I think it's an interesting exercise, and one that we can learn from, and help us think about some important matehmatical concepts from a new perspective. 

What do we truly know? After all, we are trying to mathematically analyze a painting by possibly the most famous impressionlist artist in history. 


## Gathering what we know


Let's start by collecting some pieces of infomrmation that we know.

Looking at the painting, I notice a few useful features.

1. The sun
2. A distinctive rock pinicle.
3. The horizon

Let's look at each of these in isolation.

![Measuements on the The Cliff, Étretat, Sunset](../images/2020-04-13-Deconstructing-A-Monet/Measurements.jpg)


### The Sun

Let's start with some facts:

* The sun has a radius of $6.96\times 10^8$ m
* The sun is $1.5\times 10^{11}$ m from earth.

In the image of the painting, it occupies a bounding box limited by:
* x1,y1 = 4180px, 1973px
* x2,y2 = 4330px, 2115px

I.e. it has a radius of ±73 pixels. 

Let's pause for a moment, and make a critical assumption, that it was painted with perspecrtive. 

Other works by Monet clearly have perspective, which we can directly measure, for example *The Railway Bridge at Argenteuil* which we can see below.

![The Railway Bridge at Argenteuil](../images/2020-04-13-Deconstructing-A-Monet/The_Railway_Bridge_at_Argenteuil.jpg)

If we work with this assumption, we can proceed to work out the "focal length" of the painting in pixels.


![Image Triangle](../images/2020-04-13-Deconstructing-A-Monet/Triangle.jpg)

From this, we can see that 


$\frac{R_{sun}}{D_{sun}} = \frac{r_{pixels}} {f_{pixels}}$

or 

$ f_{pixels} = \frac{r_{pixels}}{(\frac{R_{sun}}{D_{sun}})}$

In [ ]:
R_sun = 6.96* 10**8 
D_sun = 1.5*10**11
r_pixels = 73
f_pixels = (r_pixels/(R_sun/D_sun))
print(int(f_pixels))


$ f_{pixels} = 15.7 \times 10^3$

### A distinctive rock pinicle.
    

Height of the pinicle{% fn 1 %}: 
* Low tide = 54.5m
* High tide = 46.0m 
* Avg = 50.25m 

In the image, we see that:
* Pinicle Peak = 1423 px
* Pinicle Base = 2486 px
* Pinicle Height = 1063 px


Using the same geometric argument from before, we see that:

$\frac{R_{pinicle}}{D_{pinicle}} = \frac{r_{pixels}} {f_{pixels}}$

$D_{pinicle} = \frac{R_{pinicle}}{(\frac{r_{pixels}} {f_{pixels}})}$


In [ ]:
R_pinicle = 50.25
r_pixels = 1063
D_pinicle = int(R_pinicle/(r_pixels / (f_pixels)))

print(D_pinicle)

So assuming the pinicle is approximately 743m from the painter. 

### The Horizon

Let's look at the last point we can clearly measure in the painting, the horizon line.

The distance $d$ to the horizon is:
${\displaystyle d\approx {\sqrt {2\,h\,r}}\,}$

Where:
* $r$ is the radius of the earth ($6,371 \times 10^3$m)
* $h$ is the height of the observer

Assuming that $h$ is at least 2m, then the distance to the horizon is at least 5km.


![Horizon Triangle](../images/2020-04-13-Deconstructing-A-Monet/Horizon.jpg)

horizon = 2350

r_intersection = 136




2486 - 2350








## Broad Brushstrokes



## Putting it all together

{{ '"Several early guidebooks gave the Needle a height of 225 feet (69 meters) or 230 feet (70 meters), exaggerated values adopted by the majority of recent authors.

Our results showed that even near low tide the top of the Needle stands only about 179 feet (54.5 meters) above the exposed base. 

When the highest possible tides occur, 151 feet (46 meters) of the Needle remain visible above the waves. 

This photograph shows a view from the Jambourg beach near low tide, with the high-water mark clearly visible on the Needle about 28 feet (8.5 meters) above the exposed base."' | fndetail: 1 }}